In [81]:
!git clone https://github.com/m0chi1216/hlab2023-nlp100.git

Cloning into 'hlab2023-nlp100'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 108 (delta 35), reused 96 (delta 24), pack-reused 0
Receiving objects: 100% (108/108), 16.99 MiB | 24.58 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [82]:
cd hlab2023-nlp100

/content/hlab2023-nlp100/hlab2023-nlp100


In [83]:
!git checkout -b suzuki origin/suzuki

Branch 'suzuki' set up to track remote branch 'suzuki' from 'origin'.
Switched to a new branch 'suzuki'


In [84]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
!unzip NewsAggregatorDataset.zip

--2023-05-29 11:19:08--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  35.3MB/s    in 0.8s    

2023-05-29 11:19:09 (35.3 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203/29224203]

Archive:  NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [85]:
!sed -e 's/"/'\''/g' ./newsCorpora.csv > ./newsCorpora_re.csv


In [86]:
with open("readme.txt","r")as fp:
  words=fp.read()
  print(words)
  fp.close()

SUMMARY: Dataset of references (urls) to news web pages

DESCRIPTION: Dataset of references to news web pages collected from an online aggregator in the period from March 10 to August 10 of 2014. The resources are grouped into clusters that represent pages discussing the same news story. The dataset includes also references to web pages that point (has a link to) one of the news page in the collection.

TAGS: web pages, news, aggregator, classification, clustering

LICENSE: Public domain - Due to restrictions on content and use of the news sources, the corpus is limited to web references (urls) to web pages and does not include any text content. The references have been retrieved from the news aggregator through traditional web browsers. 

FILE ENCODING: UTF-8

FORMAT: Tab delimited CSV files. 

DATA SHAPE AND STATS: 422937 news pages and divided up into:

152746 	news of business category
108465 	news of science and technology category
115920 	news of business category
 45615 	news of

In [87]:
!pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [88]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('./newsCorpora_re.csv',header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])
df=df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']),['TITLE', 'CATEGORY']]

train,valid=train_test_split(df,test_size=0.2,random_state=1,shuffle=True,stratify=df["CATEGORY"])
valid,test=train_test_split(valid,test_size=0.5,random_state=1,shuffle=True,stratify=valid["CATEGORY"])

train.to_csv('./train.txt', sep='\t', index=False)
valid.to_csv('./valid.txt', sep='\t', index=False)
test.to_csv('./test.txt', sep='\t', index=False)

print('学習データ')
print(train['CATEGORY'].value_counts())
print('検証データ')
print(valid['CATEGORY'].value_counts())
print('評価データ')
print(test['CATEGORY'].value_counts())

学習データ
b    4501
e    4235
t    1220
m     728
Name: CATEGORY, dtype: int64
検証データ
b    563
e    530
t    152
m     91
Name: CATEGORY, dtype: int64
評価データ
b    563
e    529
t    153
m     91
Name: CATEGORY, dtype: int64


In [89]:
import string
import re

def preprocessing(text):
  table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
  text = text.translate(table)  # 記号をスペースに置換
  text = text.lower()  # 小文字化
  text = re.sub('[0-9]+', '0', text)  # 数字列を0に置換

  return text

In [90]:
df=pd.concat([train,valid,test])
df.reset_index(drop=True,inplace=True)
df["TITLE"]=df["TITLE"].map(lambda x:preprocessing(x))
print(df.head())

                                               TITLE CATEGORY
0  new microsoft ceo nadella impresses wall stree...        b
1  argentina won t make june 0 debt payment after...        b
2  is the ipad a big iphone or a small mac  or is...        t
3  strong jobs report raises a question  how much...        b
4  tougher leverage caps for us banks set for reg...        b


In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer

train_valid = df[:len(train) + len(valid)]
test = df[len(train) + len(valid):]

vec_tfidf=TfidfVectorizer(min_df=30)
X_train_valid = vec_tfidf.fit_transform(train_valid['TITLE']) 
X_test = vec_tfidf.transform(test['TITLE'])

X_train_valid = pd.DataFrame(X_train_valid.toarray(),columns=vec_tfidf.get_feature_names_out())
X_test = pd.DataFrame(X_test.toarray(), columns=vec_tfidf.get_feature_names_out())

X_train=X_train_valid[:len(train)]
X_valid=X_train_valid[len(train):]

print(X_train.head())


   0th  about  action  actor  adds  affirms  africa     after  again  against  \
0  0.0    0.0     0.0    0.0   0.0      0.0     0.0  0.000000    0.0      0.0   
1  0.0    0.0     0.0    0.0   0.0      0.0     0.0  0.268464    0.0      0.0   
2  0.0    0.0     0.0    0.0   0.0      0.0     0.0  0.000000    0.0      0.0   
3  0.0    0.0     0.0    0.0   0.0      0.0     0.0  0.000000    0.0      0.0   
4  0.0    0.0     0.0    0.0   0.0      0.0     0.0  0.000000    0.0      0.0   

   ...  yen  yet  yields  york  you  young  your  yuan  zac  zone  
0  ...  0.0  0.0     0.0   0.0  0.0    0.0   0.0   0.0  0.0   0.0  
1  ...  0.0  0.0     0.0   0.0  0.0    0.0   0.0   0.0  0.0   0.0  
2  ...  0.0  0.0     0.0   0.0  0.0    0.0   0.0   0.0  0.0   0.0  
3  ...  0.0  0.0     0.0   0.0  0.0    0.0   0.0   0.0  0.0   0.0  
4  ...  0.0  0.0     0.0   0.0  0.0    0.0   0.0   0.0  0.0   0.0  

[5 rows x 687 columns]


In [92]:
from sklearn.linear_model import LogisticRegression

lg = LogisticRegression(random_state=123, max_iter=10000)
lg.fit(X_train, train['CATEGORY'])

LogisticRegression(max_iter=10000, random_state=123)

In [93]:
import numpy as np
def score(lg,x):
  return[np.max(lg.predict_proba(x),axis=1),lg.predict(x)]

In [94]:
train_predict=score(lg,X_train)
test_predict=score(lg,X_test)
print(train_predict)

[array([0.56948183, 0.87281235, 0.63563146, ..., 0.85853555, 0.60660315,
       0.72324666]), array(['b', 'b', 'e', ..., 'b', 'e', 'e'], dtype=object)]


In [95]:
from sklearn.metrics import accuracy_score

train_accuracy=accuracy_score(train["CATEGORY"],train_predict[1])
test_accuracy=accuracy_score(test["CATEGORY"],test_predict[1])

print(f"学習データの正解率={0}.3f",train_accuracy)
print(f"テストデータの正解率={0}.3f",test_accuracy)

学習データの正解率=0.3f 0.8675589666791463
テストデータの正解率=0.3f 0.8413173652694611


In [96]:
!git status

On branch suzuki
Your branch is up to date with 'origin/suzuki'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	2pageSessions.csv
	NewsAggregatorDataset.zip
	__MACOSX/
	newsCorpora.csv
	newsCorpora_re.csv
	readme.txt
	test.txt
	train.txt
	valid.txt

nothing added to commit but untracked files present (use "git add" to track)


In [97]:
!git add .

In [101]:
!git config --global user.email "riors2020@gmail.com"
!git config --global user.name "rio-user11"

In [102]:
!git commit -m “commit2”


[suzuki 0df0188] “commit2”
 11 files changed, 874807 insertions(+)
 create mode 100644 2pageSessions.csv
 create mode 100644 NewsAggregatorDataset.zip
 create mode 100644 __MACOSX/._2pageSessions.csv
 create mode 100644 __MACOSX/._newsCorpora.csv
 create mode 100644 __MACOSX/._readme.txt
 create mode 100644 newsCorpora.csv
 create mode 100644 newsCorpora_re.csv
 create mode 100644 readme.txt
 create mode 100644 test.txt
 create mode 100644 train.txt
 create mode 100644 valid.txt
